<a href="https://colab.research.google.com/github/DhruvSharmaaa05/DhruvSharmaaa05.github.io/blob/main/Monte_Carlo_Option_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance --quiet

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import yfinance as yf
from datetime import datetime

np.random.seed(42)
%matplotlib inline


In [4]:
def black_scholes_call(S0, K, r, sigma, T):
    """
    Black–Scholes price for a European call option.
    """
    if T <= 0 or sigma <= 0:
        return max(S0 - K, 0.0)

    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price


def black_scholes_put(S0, K, r, sigma, T):
    """
    Black–Scholes price for a European put option.
    """
    if T <= 0 or sigma <= 0:
        return max(K - S0, 0.0)

    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    return put_price


def monte_carlo_european_option(S0, K, r, sigma, T, n_sims=50_000, option_type="call"):
    """
    Monte Carlo pricing for European call/put using pure NumPy.
    """
    S0 = float(S0)
    K = float(K)
    r = float(r)
    sigma = float(sigma)
    T = float(T)

    Z = np.random.normal(0.0, 1.0, int(n_sims))
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    if option_type.lower() == "call":
        payoffs = np.maximum(ST - K, 0.0)
    elif option_type.lower() == "put":
        payoffs = np.maximum(K - ST, 0.0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    discounted = np.exp(-r * T) * payoffs
    price = float(discounted.mean())
    std_err = float(discounted.std(ddof=1) / np.sqrt(n_sims))
    return price, std_err

In [5]:
def get_live_price_and_vol(ticker, lookback_days=60):
    """
    Fetch latest price and estimate annualized volatility from past data.
    Cleans NaNs to avoid issues.
    """
    data = yf.download(
        ticker,
        period=f"{lookback_days}d",
        interval="1d",
        progress=False,
        auto_adjust=False
    )

    if data.empty:
        raise ValueError(f"No data found for ticker '{ticker}'. Check the symbol or lookback_days.")

    # Prefer Adj Close, fallback to Close
    if "Adj Close" in data.columns:
        prices = data["Adj Close"].to_numpy(dtype="float64")
    else:
        prices = data["Close"].to_numpy(dtype="float64")

    # 🔹 Remove NaNs from prices
    prices = prices[~np.isnan(prices)]

    if len(prices) < 2:
        raise ValueError("Not enough clean price data to compute volatility. Try increasing lookback_days.")

    # Last price as S0
    S0 = float(prices[-1])

    # Daily log returns
    log_returns = np.diff(np.log(prices))

    # 🔹 Remove any NaNs from returns too (extra safety)
    log_returns = log_returns[~np.isnan(log_returns)]

    if len(log_returns) == 0:
        raise ValueError("Returns are all NaN or zero. Cannot compute volatility.")

    # Annualized volatility
    sigma = float(log_returns.std(ddof=1) * np.sqrt(252))

    return S0, sigma


In [6]:
def time_to_maturity(expiry_str):
    """
    Convert expiry date string 'YYYY-MM-DD' to T (in years).
    """
    today = datetime.today().date()
    expiry = datetime.strptime(expiry_str, "%Y-%m-%d").date()

    days = (expiry - today).days
    days = max(days, 0)  # avoid negative

    T = days / 365.0
    return T


In [7]:
def price_option_live(
    ticker,
    K,
    r,
    expiry_str,
    option_type="call",
    n_sims=50_000,
    lookback_days=60
):
    """
    Full pipeline:
    1. Fetch live price + estimate volatility
    2. Compute time to maturity from expiry date
    3. Price option via Monte Carlo + Black–Scholes
    """
    # 1. Live data
    S0, sigma = get_live_price_and_vol(ticker, lookback_days)

    # 2. Time to maturity
    T = time_to_maturity(expiry_str)

    # 3. Monte Carlo
    mc_price, mc_err = monte_carlo_european_option(S0, K, r, sigma, T, n_sims, option_type)

    # 4. Black–Scholes (only valid for European)
    if option_type.lower() == "call":
        bs_price = black_scholes_call(S0, K, r, sigma, T)
    else:
        bs_price = black_scholes_put(S0, K, r, sigma, T)

    return {
        "ticker": ticker,
        "S0": S0,
        "sigma": sigma,
        "T": T,
        "K": K,
        "r": r,
        "option_type": option_type,
        "n_sims": n_sims,
        "mc_price": mc_price,
        "mc_err": mc_err,
        "bs_price": bs_price,
    }


In [8]:
ticker = "RELIANCE.NS"      # or "AAPL"
K = 2800.0
r = 0.06
expiry_str = "2026-01-29"
option_type = "call"
n_sims = 100_000

result = price_option_live(
    ticker=ticker,
    K=K,
    r=r,
    expiry_str=expiry_str,
    option_type=option_type,
    n_sims=n_sims,
    lookback_days=90
)

print("=== Live Inputs ===")
print(f"Ticker: {result['ticker']}")
print(f"Spot Price S0: {result['S0']:.2f}")
print(f"Estimated Volatility σ: {result['sigma']:.4f}")
print(f"Time to Maturity T: {result['T']:.4f} years")
print(f"Strike K: {result['K']:.2f}")
print(f"Risk-free rate r: {result['r']:.4f}")
print(f"Simulations: {result['n_sims']:,}")
print(f"Option Type: {result['option_type']}")

print("\n=== Prices ===")
ci_low = result['mc_price'] - 1.96 * result['mc_err']
ci_high = result['mc_price'] + 1.96 * result['mc_err']

print(f"Monte Carlo Price: {result['mc_price']:.2f}")
print(f"  95% CI: [{ci_low:.2f}, {ci_high:.2f}]")
print(f"Black–Scholes Price: {result['bs_price']:.2f}")


=== Live Inputs ===
Ticker: RELIANCE.NS
Spot Price S0: 1528.50
Estimated Volatility σ: 0.1529
Time to Maturity T: 0.1397 years
Strike K: 2800.00
Risk-free rate r: 0.0600
Simulations: 100,000
Option Type: call

=== Prices ===
Monte Carlo Price: 0.00
  95% CI: [0.00, 0.00]
Black–Scholes Price: 0.00


In [9]:
def plot_live_terminal_distribution(result):
    S0 = float(result["S0"])
    K = float(result["K"])
    r = float(result["r"])
    sigma = float(result["sigma"])
    T = float(result["T"])
    n_sims = int(result["n_sims"])

    # Debug print (optional, helps you see values)
    print("Debug - S0:", S0, "K:", K, "r:", r, "sigma:", sigma, "T:", T, "n_sims:", n_sims)

    # 🔹 Safety checks
    if np.isnan(S0) or np.isnan(sigma) or np.isnan(T):
        raise ValueError(f"One of the inputs is NaN. Got S0={S0}, sigma={sigma}, T={T}")

    if n_sims <= 0:
        raise ValueError(f"n_sims must be > 0, got {n_sims}")

    Z = np.random.normal(0.0, 1.0, n_sims)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    if np.all(np.isnan(ST)):
        raise ValueError("All simulated terminal prices ST are NaN. Check inputs.")

    plt.figure(figsize=(8, 5))
    plt.hist(ST, bins=60, density=True, alpha=0.7)
    plt.axvline(K, linestyle="--")
    plt.title(f"Distribution of $S_T$ for {result['ticker']} (live inputs)")
    plt.xlabel("$S_T$")
    plt.ylabel("Density")
    plt.show()
